## Purpose of first model
The purpose of the FINAL model is to do topic modeling of both hits in the US and the UK and comparing them.
However, this first, simple, prelimiary model will do just the US, dropping the NaN rows (currently trying to find ways to fill them in the webscraper module).



In [56]:
import pickle
import pandas as pd

import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.decomposition import LatentDirichletAllocation

In [57]:
file = open('../Webscraper/us_pops_raw', 'rb')
us_pops = pickle.load(file)
us_pops.dropna(inplace=True)
us_pops = us_pops['Lyrics']

In [58]:
us_pops

0       I know this pain (I know this pain)\nWhy do yo...
1       Lay a whisper on my pillow\nLeave the winter o...
2       It's been seven hours and fifteen days\nSince ...
3       Yeah, Spyderman and Freeze in full effect\nUh-...
4       Strike a pose\nStrike a pose\nVogue (vogue, vo...
                              ...                        
2750    We've been to both Carolinas\nSeen a big Monta...
2751    I'm jealous of the blue jeans that you're wear...
2752    I'm a motherfuckin' train wreck\nI don't wanna...
2753    There's somethin' in the way you roll your eye...
2754    Man, what? (Haha)\nThis shit funny, one sec\nO...
Name: Lyrics, Length: 2671, dtype: object

In [59]:
# only keep alphanumeric, remove newlines, and make everything lowercase
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
rm_newline = lambda x: re.sub("\n", ' ',x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

us_pops_nopunc = us_pops.map(alphanumeric).map(rm_newline).map(punc_lower)

In [60]:
# lemmatize. do this by turning every line into a list of words, and concat back to full lyrics
us_pops_list = us_pops_nopunc.apply(lambda x: x.split(' '))

def lemmatize(list_of_words):
    lmt = WordNetLemmatizer()
    strg = ''
    for word in list_of_words:
        lem_word = lmt.lemmatize(word)
        if len(lem_word) > 2:
            strg += lem_word + ' '
    return strg
        

us_pops_clean = us_pops_list.apply(lemmatize)

In [61]:
cv = CountVectorizer(
    stop_words = 'english',
    # max_df = 0.1,
    # min_df = 0.008
)
X = cv.fit_transform(us_pops_clean)
doc_term_matrix = pd.DataFrame(X.toarray(), columns=cv.get_feature_names())

In [62]:
doc_term_matrix

,aaa,aaaa,aaaaaa,aaaaaaaaaaaaaaaaaaaa,aaaaaah,aaaah,aaaant,aagh,aah,aaherra,...,에에에,여유를,울퉁불퉁한,웬만한,이때다,인간적인,점잖아,정숙해,지금부터,한잔의
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2667,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2668,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2669,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
from sklearn.feature_extraction import text

In [64]:
stop_words_added = text.ENGLISH_STOP_WORDS.union(['huh','woo','whoa','hey','hold'])
# stop_words_custom = frozenset({'huh','woo','whoa','hey','hold'})

In [75]:
tfidf =  TfidfVectorizer(
    stop_words = stop_words_added,
    max_df = 0.2,
    min_df = 0.0005
)
Y = tfidf.fit_transform(us_pops_clean)
doc_term_matrix2 = pd.DataFrame(Y.toarray(), columns=tfidf.get_feature_names())


In [76]:
doc_term_matrix2

,aaa,aah,aaliyah,aback,abandoned,abashed,abc,abdul,abeat,abel,...,zulu,zurück,çünkü,étais,était,être,über,üstünde,şey,şimdi
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.046599,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2667,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2668,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2669,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### LSA below

In [67]:
lsa = TruncatedSVD(5)
doc_topic = lsa.fit_transform(doc_term_matrix2)
lsa.explained_variance_ratio_

array([0.0036137 , 0.0097805 , 0.00633444, 0.00525201, 0.0052064 ])

In [68]:
topic_word = pd.DataFrame(
    lsa.components_.round(3), 
    index = ['topic1','topic2','topic3','topic4','topic5'],
    columns = tfidf.get_feature_names()
    )
topic_word

,able,account,ace,ache,act,actin,acting,action,actually,add,...,yep,yes,yesterday,york,young,younger,youth,yuh,yup,zone
topic1,0.008,0.005,0.003,0.003,0.029,0.012,0.007,0.007,0.006,0.004,...,0.008,0.059,0.016,0.014,0.052,0.005,0.002,0.006,0.005,0.009
topic2,-0.007,0.008,0.001,-0.002,0.025,0.009,0.003,0.005,-0.003,0.001,...,0.005,-0.042,-0.018,0.014,0.032,-0.003,-0.001,0.002,0.007,0.007
topic3,-0.007,-0.001,0.001,-0.003,-0.006,-0.003,-0.002,-0.001,-0.007,-0.002,...,-0.002,0.006,-0.015,0.001,0.010,0.000,-0.002,-0.000,0.002,0.006
topic4,0.001,0.000,-0.000,0.000,-0.002,0.000,-0.000,0.001,0.002,0.000,...,0.003,0.003,0.000,0.002,-0.004,0.001,0.001,0.004,0.000,0.000
topic5,0.001,0.001,0.001,0.000,-0.004,-0.005,-0.002,-0.004,-0.005,-0.002,...,-0.005,-0.033,0.016,-0.001,0.018,-0.002,-0.001,-0.002,-0.001,-0.007


In [69]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [70]:
display_topics(lsa, cv.get_feature_names(), 10)


Topic  0
anyour, authentic, alligator, acadian, arguing, ascribe, abyss, anfractuosities, alive, aboo

Topic  1
anyour, acadian, ascribe, alive, abolishing, antenna, allfaulters, andrea, alwagers, aluminum

Topic  2
authentic, accompany, afore, apostrophe, armschair, abbess, alligator, alert, ating, arzt

Topic  3
archaic, approvingly, arzt, aflash, accompany, askit, angelic, afore, armschair, airy

Topic  4
authentic, archaic, anyour, acadian, abolishing, alive, abstract, aristocracy, ascribe, algiers


### NMF below

In [108]:
nmf_model = NMF(5)
doc_topic2 = nmf_model.fit_transform(doc_term_matrix2)

In [117]:
doc_topic_mat = pd.DataFrame(
    doc_topic2.round(3),
    columns = ['topic1','topic2','topic3','topic4','topic5']
)
doc_topic_mat

,topic1,topic2,topic3,topic4,topic5
0,0.051,0.000,0.000,0.011,0.043
1,0.070,0.000,0.005,0.009,0.000
2,0.041,0.002,0.000,0.000,0.009
3,0.018,0.011,0.002,0.018,0.004
4,0.002,0.000,0.000,0.082,0.000
...,...,...,...,...,...
2666,0.057,0.000,0.000,0.000,0.026
2667,0.042,0.000,0.005,0.060,0.000
2668,0.011,0.003,0.004,0.011,0.048
2669,0.052,0.006,0.135,0.000,0.011


In [112]:
topic_word = pd.DataFrame(
    nmf_model.components_.round(3),
    columns = tfidf.get_feature_names()
    )
topic_word

,aaa,aah,aaliyah,aback,abandoned,abashed,abc,abdul,abeat,abel,...,zulu,zurück,çünkü,étais,était,être,über,üstünde,şey,şimdi
0,0.004,0.019,0.000,0.0,0.004,0.0,0.000,0.001,0.001,0.001,...,0.000,0.000,0.0,0.0,0.0,0.0,0.002,0.0,0.0,0.0
1,0.000,0.000,0.003,0.0,0.000,0.0,0.001,0.000,0.000,0.000,...,0.000,0.003,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0
2,0.000,0.000,0.000,0.0,0.000,0.0,0.000,0.000,0.000,0.000,...,0.000,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0
3,0.000,0.012,0.001,0.0,0.000,0.0,0.000,0.000,0.000,0.000,...,0.001,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0
4,0.000,0.000,0.002,0.0,0.000,0.0,0.002,0.000,0.000,0.000,...,0.000,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0


In [110]:
display_topics(nmf_model, tfidf.get_feature_names(), 15)


Topic  0
dream, believe, gone, forever, long, remember, wish, true, fall, inside, home, face, place, better, stay

Topic  1
nigga, bitch, shit, fuck, ayy, money, gon, lil, hoe, real, big, hit, fuckin, pussy, dick

Topic  2
tonight, alright, waiting, tomorrow, fight, party, tight, kiss, inside, gotta, drink, dancing, boo, end, broken

Topic  3
dance, body, shake, rock, everybody, party, stop, turn, floor, play, music, round, roll, club, beat

Topic  4
really, gotta, bad, somebody, woman, real, friend, yes, lover, try, babe, stay, sorry, talk, care


### LDA Below

In [74]:
# # for TF DTM
# lda_tf = LatentDirichletAllocation(n_components=20, random_state=0)
# lda_tf.fit(doc_term_matrix)

# # for TFIDF DTM
# lda_tfidf = LatentDirichletAllocation(n_components=20, random_state=0)
# lda_tfidf.fit(doc_term_matrix2)